# Testing code

# Importing stuff

In [42]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import os.path as osp
import random

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm_notebook

from nlpclass.config import model_config
from nlpclass.data.data_utils import TranslationDataset, text_collate_func
from nlpclass.models.evaluation_utils import bleu_eval, output_to_translations
from nlpclass.models.models import DecoderRNN, EncoderCNN, EncoderRNN, TranslationModel
from nlpclass.models.training_utils import load_data

In [44]:
CURRENT_PATH = os.getcwd()
DATA_DIR = osp.join(CURRENT_PATH, '..', 'data')
MODEL_DIR = osp.join(CURRENT_PATH, '..','models')

# Data

In [85]:
data, data_loaders, max_length = load_data('vi', batch_size=2, subsample=0.025)

Counting words...
Counted words:
eng 6709
vi 7674
Counting words...
Counted words:
eng 3562
vi 3678
Counting words...
Counted words:
eng 3361
vi 3518


In [5]:
encoder = EncoderCNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2).to(model_config.device)
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=128,
    output_size=data['train'].output_lang.n_words,
    attention=False).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [46]:
encoder = EncoderRNN(
    data['train'].input_lang.n_words,
    embedding_size=100,
    hidden_size=128,
    num_layers=2,
    dropout=0.2,
    bidirectional=True).to(model_config.device)
if encoder.bidirectional:
    multiplier = 2
else:
    multiplier = 1
decoder = DecoderRNN(
    embedding_size=100,
    hidden_size=multiplier * 128,
    output_size=data['train'].output_lang.n_words,
    attention=True).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=0.5).to(model_config.device)

In [47]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)

In [48]:
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [49]:
def calc_loss(logits, target, criterion):
    logits_flat = logits.view(-1, logits.size(-1))
    target_flat = target.view(-1, 1).squeeze()
    return criterion(logits_flat, target_flat)

In [50]:
def evaluate(model, data, data_loaders, dataset_type='dev', max_batches=100):
    model.eval()
    with torch.no_grad():
        original_strings = []
        translated_strings = []
        for i, batch in enumerate(data_loaders[dataset_type]):
            if i > max_batches:
                break
            logits = translation_model(batch)
            epoch_loss = calc_loss(logits, batch['target'], criterion)
            original = output_to_translations(batch['target'], data['train'])
            translations = output_to_translations(model.greedy(batch), data['train'])
            original_strings.extend(original)
            translated_strings.extend(translations)
        bleu = bleu_eval(original_strings, translated_strings)
        model.train()
        print(epoch_loss)
        print(bleu)
        
        return original_strings, translated_strings

In [17]:
for i in tqdm_notebook(range(25)):
    for batch in data_loaders['train']:
        optimizer.zero_grad()
        logits = translation_model(batch)
        loss = calc_loss(logits, batch['target'], criterion)
        loss.backward()
        
        encoder_norm = 0
        for p in translation_model.encoder.parameters():
            param_norm = p.grad.data.norm(2)
            encoder_norm += param_norm.item() ** 2
        decoder_norm = 0
        for p in translation_model.decoder.parameters():
            param_norm = p.grad.data.norm(2)
            decoder_norm += param_norm.item() ** 2
            
        print(encoder_norm, decoder_norm, loss.item())
            
        clip_grad_norm_(filter(lambda p: p.requires_grad,
                                   translation_model.parameters()), 5.0)
        
        optimizer.step() 
    original, translation = evaluate(translation_model, data, data_loaders, dataset_type='train')

0.0020137496824000926 4.634426597858985 5.626060962677002
0.013517843523675067 107.33386255863229 12.328470230102539
0.48494471422290086 222.0302262670534 15.045141220092773
30.391699882161944 214.54025889667741 10.102069854736328
0.4999193563602579 3.259551601435442 7.100986003875732
2.4976484312463882 3.00420777392432 6.747938632965088
0.15120426317697222 3.2237099564328795 6.767729759216309
0.21003912480436002 4.4115080652282606 6.9724836349487305
0.9085347633240615 2.9892800583846686 7.226278305053711
152.3248277138639 545.550505056196 14.625187873840332
0.1274045836639269 11.903812258313298 7.20048189163208
1.8443979550091911 3.2978176538244597 7.7557477951049805
0.20664004741306952 1.3343666424972969 7.667934417724609
0.092596585404913 1.8405205211811728 7.21620512008667
0.09017671927511317 2.118393123799774 7.055311679840088
0.049907650533802174 2.6228875314903672 6.981229305267334
0.043321533647958016 1.7091123785435929 6.3538594245910645
0.05181700890728001 3.7120551973889384 

0.005066078404559122 0.9399914171559608 5.947779655456543
0.009557955448134833 1.458418145081454 6.538112163543701
0.007028175551751372 1.1404207871505092 6.5298357009887695
0.01995761000481369 0.9203472766135045 6.701842308044434
0.007616119358221397 1.320857260665912 5.865251064300537
0.010199047609220614 1.0271163162594674 5.690906524658203
0.010846928040286403 0.7417836377620955 5.936854362487793
0.0058806240524687595 0.9846039297058174 6.61268424987793
0.005462618259263431 1.0709079491629383 5.701878070831299
0.004122173196795206 0.9372017552358015 6.307410717010498
0.002715935338956546 0.6911030680388788 6.213427543640137
0.006173176831278642 1.1854937319657601 6.815855979919434
0.003774298579827752 1.3353050030469464 7.183228492736816
0.004010815344478557 1.870776661222798 7.045688152313232
0.0036580602171487615 1.2312393664152097 6.1077961921691895
0.047271054072556606 1.0185015957753891 5.933896541595459
0.009984990338158698 1.051970161676569 6.969225883483887
0.00739492844892

KeyboardInterrupt: 

In [101]:
#x = next(iter(data_loaders['train']))
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.greedy(x), data['train'])
print(bleu_eval(original, translations))
print(list(zip(translations[0:2], original[0:2])))

tensor([0, 0], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([37, 37], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([234,  33], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([411,  13], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([33, 33], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([704,  13], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([12, 33], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([ 7, 13], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([10, 33], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([12, 13], device='cuda:0') torch.Size([2])
torch.Size([2, 1, 100]) torch.Size([2, 1, 256])
tensor([ 7, 64],

In [60]:
translation_model.beam_size = 1

In [61]:
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.beam_search(x), data['train'])
print(bleu_eval(original, translations))
print(list(zip(translations[0:3], original[0:3])))

[0.93794316]
[0.44554293]
[0.28218982]
[0.20106924]
[0.15281946]
[0.12098749]
[0.09852279]
[0.08190073]
[0.06916371]
[0.05913762]
[0.05107569]
[0.0444804]
[0.03900763]
[0.03441185]
[0.0305132]
[0.02717704]
[0.02430056]
[0.02180397]
[0.01962445]
[0.01771186]
[0.01602574]
[0.01453308]
[0.01320674]
[0.01202417]
[0.01096653]
[0.01001796]
[0.00916498]
[0.00839616]
[0.00770165]
[0.00707299]
[0.00650288]
[0.00598495]
[0.00551366]
[0.00508414]
[0.00469215]
[0.00433391]
[0.00400612]
[0.00370583]
[0.00343042]
[0.00317757]
[0.0029452]
[0.00273144]
[0.00253464]
[0.00235328]
[0.00218604]
[0.00203168]
[0.00188911]
[0.00175735]
[0.00163548]
[0.0015227]
[0.00141827]
[0.0013215]
[0.0012318]
[0.00114859]
[0.00107137]
[0.00099967]
[0.00093306]
[0.00087117]
[0.00081362]
[0.00076009]
[0.00071028]
[0.00066391]
[0.00062073]
[0.00058051]
[0.00054302]
[0.00050808]
[0.00047549]
[0.00044509]
[0.00041673]
[0.00039025]
[0.00036554]
[0.00034245]
[0.00032089]
[0.00030074]
[0.0002819]
[0.0002643]
[0.00024783]
[0.0002

[0.00414155]
[0.00392957]
[0.00372994]
[0.0035418]
[0.0033644]
[0.00319703]
[0.00303902]
[0.00288978]
[0.00274874]
[0.00261539]
[0.00248925]
[0.00236988]
[0.00225685]
[0.0021498]
[0.00204835]
[0.00195219]
[0.00186099]
[0.00177447]
[0.00169237]
[0.00161442]
[0.0015404]
[0.00147008]
[0.00140325]
[0.00133973]
[0.00127933]
[0.00122189]
[0.00116723]
[0.00111522]
[0.00106571]
[0.00101857]
[0.00097368]
[0.00093091]
[0.00089017]
[0.00085133]
[0.00081431]
[0.00077901]
[0.00074535]
[0.00071324]
[0.00068261]
[0.00065338]
[0.00062548]
[0.00059885]
[0.00057342]
[0.00054914]
[0.00052595]
[0.0005038]
[0.00048263]
[0.00046241]
[0.00044308]
[0.0004246]
[0.00040694]
[0.96910384]
[0.4724686]
[0.3071244]
[0.22459921]
[0.17519868]
[0.14235811]
[0.11897839]
[0.10150998]
[0.08798099]
[0.0772083]
[0.06843905]
[0.06117134]
[0.05505776]
[0.04985015]
[0.04536657]
[0.04147055]
[0.03805779]
[0.03504714]
[0.03237458]
[0.02998889]
[0.02784862]
[0.02591988]
[0.02417466]
[0.02258966]
[0.02114528]
[0.01982498]
[0.01861

[0.00808368]
[0.00796727]
[0.00785348]
[0.00774223]
[0.00763344]
[0.00752702]
[0.00742292]
[0.00732105]
[0.00722134]
[0.00712374]
[0.96483234]
[0.46975939]
[0.30495645]
[0.22271666]
[0.17349873]
[0.14078899]
[0.11751018]
[0.10012377]
[0.08666391]
[0.07595116]
[0.06723499]
[0.06001508]
[0.05394509]
[0.04877764]
[0.04433138]
[0.04047027]
[0.03709034]
[0.03411071]
[0.03146758]
[0.02910989]
[0.02699634]
[0.02509315]
[0.02337241]
[0.02181091]
[0.02038913]
[0.01909057]
[0.0179012]
[0.01680898]
[0.01580357]
[0.01487598]
[0.01401841]
[0.01322403]
[0.01248687]
[0.01180164]
[0.01116366]
[0.01056881]
[0.01001337]
[0.00949406]
[0.00900792]
[0.0085523]
[0.0081248]
[0.00772326]
[0.00734573]
[0.00699044]
[0.00665577]
[0.00634025]
[0.00604255]
[0.00576143]
[0.00549578]
[0.00524456]
[0.00500683]
[0.00478171]
[0.0045684]
[0.00436616]
[0.00417431]
[0.0039922]
[0.00381926]
[0.00365494]
[0.00349872]
[0.00335015]
[0.00320877]
[0.00307419]
[0.00294602]
[0.0028239]
[0.00270751]
[0.00259653]
[0.00249067]
[0.00

[0.01038144]
[0.01017437]
[0.00997358]
[0.00977882]
[0.00958981]
[0.00940632]
[0.00922812]
[0.009055]
[0.00888674]
[0.00872317]
[0.00856409]
[0.00840932]
[0.00825872]
[0.00811211]
[0.00796934]
[0.00783029]
[0.0076948]
[0.00756276]
[0.00743403]
[0.00730851]
[0.00718608]
[0.00706663]
[0.00695007]
[0.00683629]
[0.0067252]
[0.00661672]
[0.00651076]
[0.00640724]
[0.00630607]
[0.00620719]
[0.00611053]
[0.00601601]
[0.00592356]
[0.97782114]
[0.47807073]
[0.31164749]
[0.22855337]
[0.17878882]
[0.14568735]
[0.12210622]
[0.10447408]
[0.09080689]
[0.07991421]
[0.07103855]
[0.0636749]
[0.05747366]
[0.05218515]
[0.04762626]
[0.04365967]
[0.0401804]
[0.03710679]
[0.03437439]
[0.03193165]
[0.02973684]
[0.02775583]
[0.02596042]
[0.02432714]
[0.02283626]
[0.02147111]
[0.02021747]
[0.01906317]
[0.01799774]
[0.01701208]
[0.01609829]
[0.01524945]
[0.01445949]
[0.01372305]
[0.0130354]
[0.01239232]
[0.01179006]
[0.01122527]
[0.01069495]
[0.01019638]
[0.00972713]
[0.00928501]
[0.008868]
[0.00847431]
[0.00810

In [27]:
x = next(iter(data_loaders['train']))
input_seq = x['input']
target_seq = x['target']
input_length = x['input_length']
target_length = x['target_length']

encoded_input, hidden = translation_model.encoder.forward(input_seq, input_length)

In [51]:
x = next(iter(data_loaders['train']))

optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-2)

for i in range(100):
    optimizer.zero_grad()
    logits = translation_model(x)
    loss = calc_loss(logits, x['target'], criterion)
    print(loss)
    loss.backward()
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(9.2197, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(8.7238, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.8933, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.3296, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.0538, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.0806, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.9268, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.8758, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.7349, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.6699, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.3861, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1947, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1317, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.9962, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.8494, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6858, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.6537, device='cuda:0', grad_fn=<NllLossBackward

In [29]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [30]:
for i, batch in enumerate(tqdm_notebook(data_loaders['train'])):
    if i % 500 == 0:
        evaluate(translation_model, data, data_loaders)
        #evaluate(translation_model, data, data_loaders, dataset_type='train')
    optimizer.zero_grad()
    logits = translation_model(batch)
    loss = calc_loss(logits, batch['target'], criterion)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(19.1450, device='cuda:0')
0.0008142795832761162
tensor(9.0464, device='cuda:0')
0.0033784249080402885


KeyboardInterrupt: 

In [ ]:
original_strings, translated_strings = evaluate(translation_model, data, data_loaders)

In [13]:
original_strings[0:10]

['baseball be later but of volumes all. hope. answer. only were',
 'be this. 65. know what must truth. i most',
 'games be practical everybody pins your needles all forward careful were',
 'i moment. it attack. be sorrows. ten.',
 'canadian be appointment careful laugh. no truth. something time were',
 'beautiful cupboard. hot. wine these',
 'much. but how bitter really dishonesty fed recently',
 'applied all else fed to climate husbands. matter. key',
 'but they answer. perfect. today. tape',
 'i is invented i situation getting powers your few']

In [14]:
translated_strings[0:10]

['i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the']

In [48]:
decoder_input.size()

torch.Size([16])

In [23]:
total_loss.backward()

In [30]:
predictions = translation_model.greedy(x)

In [215]:
for row in predictions.cpu().numpy():
    decoded_words = []
    for elem in row[1:]:
        decoded_words.append(data['train']['output_lang'].index2word[elem])
        if elem == model_config.EOS_token:
            break

In [167]:
yo = Variable(torch.LongTensor([model_config.SOS_token] * 8)).to(model_config.device)
yo = torch.stack((yo, topi.squeeze(), topi.squeeze()), dim=1)

In [99]:
total_loss.backward()

In [57]:
seq_range = torch.autograd.Variable(torch.LongTensor(np.repeat([2], len(x['input_length'])))).to(model_config.device)
mask = seq_range < x['input_length']
loss = -torch.gather(decoder_output, dim=1, index=input_var.unsqueeze(1)).squeeze() * mask.float()

In [66]:
loss.sum() / torch.sum(loss > 0).float()

tensor(11.4193, device='cuda:0', grad_fn=<DivBackward1>)

In [63]:
torch.sum(loss > 0).cpu().numpy()

array(8)

In [ ]:
encoder_output, encoder_hidden = encoder(x['input'], x['input_length'])

In [290]:
context = None
if decoder.attention:
    context = Variable(torch.zeros(encoder_output.size(0), encoder_output.size(2))).unsqueeze(1).to(model_config.device)

In [291]:
decoder_output, decoder_hidden, context, weights = decoder(input_var, encoder_hidden, encoder_output, context)

In [16]:
def train_model(model, optimizer, train_loader, criterion):
    model.train()
    loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch['label'])
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * \
            len(batch['label']) / len(train_loader.dataset)
    return loss_train

In [17]:
optimizer = torch.optim.Adam(translation_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
train_model(translation_model, optimizer, train_loader, criterion)

KeyError: 'label'